In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sample-data/sample_submission.csv
/kaggle/input/sample-data/train (1).csv
/kaggle/input/sample-data/test.csv


Importing the required libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from scipy.stats import chi2_contingency
from scipy.stats import mode
from scipy.stats import shapiro
from scipy import stats

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder , StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, cohen_kappa_score, log_loss, f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFE, RFECV
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import CalibrationDisplay
from sklearn.inspection import PartialDependenceDisplay
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

Importing the Datasets

In [3]:
train = pd.read_csv('/kaggle/input/sample-data/train (1).csv')
test = pd.read_csv('/kaggle/input/sample-data/test.csv')

EDA

In [4]:
train.describe()

,id,hospital_number,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,lesion_3
count,1235.0000,1.235000e+03,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000,1235.000000
mean,617.0000,9.545004e+05,38.202186,79.574089,30.054251,4.382591,49.602429,21.388016,3.290931,3832.496356,14.612146,3.577328
std,356.6581,1.356403e+06,0.788668,29.108638,16.452066,1.937357,10.535800,26.676453,1.589195,5436.733774,193.705735,88.858953
min,0.0000,5.213990e+05,35.400000,30.000000,8.000000,1.000000,23.000000,3.500000,0.100000,0.000000,0.000000,0.000000
25%,308.5000,5.288000e+05,37.800000,53.000000,18.000000,2.000000,43.000000,6.600000,2.000000,2205.000000,0.000000,0.000000
50%,617.0000,5.297770e+05,38.200000,76.000000,28.000000,4.500000,48.000000,7.500000,3.000000,2209.000000,0.000000,0.000000
75%,925.5000,5.341450e+05,38.600000,100.000000,36.000000,6.000000,57.000000,9.100000,4.300000,3205.000000,0.000000,0.000000
max,1234.0000,5.305129e+06,40.800000,184.000000,96.000000,7.500000,75.000000,89.000000,10.100000,41110.000000,3112.000000,2209.000000


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235 entries, 0 to 1234
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     1235 non-null   int64  
 1   surgery                1235 non-null   object 
 2   age                    1235 non-null   object 
 3   hospital_number        1235 non-null   int64  
 4   rectal_temp            1235 non-null   float64
 5   pulse                  1235 non-null   float64
 6   respiratory_rate       1235 non-null   float64
 7   temp_of_extremities    1196 non-null   object 
 8   peripheral_pulse       1175 non-null   object 
 9   mucous_membrane        1214 non-null   object 
 10  capillary_refill_time  1229 non-null   object 
 11  pain                   1191 non-null   object 
 12  peristalsis            1215 non-null   object 
 13  abdominal_distention   1212 non-null   object 
 14  nasogastric_tube       1155 non-null   object 
 15  naso

In [6]:
test.describe()

,id,hospital_number,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,lesion_1,lesion_2,lesion_3
count,824.000000,8.240000e+02,824.000000,824.000000,824.00000,824.000000,824.000000,824.000000,824.000000,824.000000,824.000000,824.0
mean,1646.500000,1.108357e+06,38.244539,80.229369,30.71966,4.508495,49.063350,20.796238,3.336420,3709.803398,12.391990,0.0
std,238.012605,1.555627e+06,0.785234,29.164711,17.43191,1.883464,10.450138,26.413588,1.539235,5112.931057,197.067771,0.0
min,1235.000000,5.213990e+05,36.000000,36.000000,9.00000,1.000000,23.000000,3.900000,0.100000,0.000000,0.000000,0.0
25%,1440.750000,5.287430e+05,37.800000,54.000000,18.00000,3.000000,43.000000,6.600000,2.000000,2205.000000,0.000000,0.0
50%,1646.500000,5.298085e+05,38.200000,76.000000,28.00000,4.500000,48.000000,7.500000,3.300000,2209.000000,0.000000,0.0
75%,1852.250000,5.346440e+05,38.600000,100.000000,36.00000,6.500000,55.000000,8.900000,4.300000,3205.000000,0.000000,0.0
max,2058.000000,5.305129e+06,40.800000,184.000000,96.00000,7.500000,75.000000,89.000000,10.100000,31110.000000,4300.000000,0.0


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824 entries, 0 to 823
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     824 non-null    int64  
 1   surgery                824 non-null    object 
 2   age                    824 non-null    object 
 3   hospital_number        824 non-null    int64  
 4   rectal_temp            824 non-null    float64
 5   pulse                  824 non-null    float64
 6   respiratory_rate       824 non-null    float64
 7   temp_of_extremities    789 non-null    object 
 8   peripheral_pulse       777 non-null    object 
 9   mucous_membrane        811 non-null    object 
 10  capillary_refill_time  818 non-null    object 
 11  pain                   795 non-null    object 
 12  peristalsis            805 non-null    object 
 13  abdominal_distention   802 non-null    object 
 14  nasogastric_tube       760 non-null    object 
 15  nasoga

In [8]:
train.isnull().sum()


id                         0
surgery                    0
age                        0
hospital_number            0
rectal_temp                0
pulse                      0
respiratory_rate           0
temp_of_extremities       39
peripheral_pulse          60
mucous_membrane           21
capillary_refill_time      6
pain                      44
peristalsis               20
abdominal_distention      23
nasogastric_tube          80
nasogastric_reflux        21
nasogastric_reflux_ph      0
rectal_exam_feces        190
abdomen                  213
packed_cell_volume         0
total_protein              0
abdomo_appearance         48
abdomo_protein             0
surgical_lesion            0
lesion_1                   0
lesion_2                   0
lesion_3                   0
cp_data                    0
outcome                    0
dtype: int64

In [9]:
test.isnull().sum()

id                         0
surgery                    0
age                        0
hospital_number            0
rectal_temp                0
pulse                      0
respiratory_rate           0
temp_of_extremities       35
peripheral_pulse          47
mucous_membrane           13
capillary_refill_time      6
pain                      29
peristalsis               19
abdominal_distention      22
nasogastric_tube          64
nasogastric_reflux        14
nasogastric_reflux_ph      0
rectal_exam_feces        125
abdomen                  154
packed_cell_volume         0
total_protein              0
abdomo_appearance         31
abdomo_protein             0
surgical_lesion            0
lesion_1                   0
lesion_2                   0
lesion_3                   0
cp_data                    0
dtype: int64

Removing the null values

In [10]:
#dropped null values
train = train.dropna(axis = 0)
test = test.fillna(0)
#drop unrelated columns
train = train.drop(columns = ['id','hospital_number'],axis = 1)

In [11]:
print(train.isnull().sum())
print("\n\n")
print(test.isnull().sum())

surgery                  0
age                      0
rectal_temp              0
pulse                    0
respiratory_rate         0
temp_of_extremities      0
peripheral_pulse         0
mucous_membrane          0
capillary_refill_time    0
pain                     0
peristalsis              0
abdominal_distention     0
nasogastric_tube         0
nasogastric_reflux       0
nasogastric_reflux_ph    0
rectal_exam_feces        0
abdomen                  0
packed_cell_volume       0
total_protein            0
abdomo_appearance        0
abdomo_protein           0
surgical_lesion          0
lesion_1                 0
lesion_2                 0
lesion_3                 0
cp_data                  0
outcome                  0
dtype: int64



id                       0
surgery                  0
age                      0
hospital_number          0
rectal_temp              0
pulse                    0
respiratory_rate         0
temp_of_extremities      0
peripheral_pulse         0
mucous_membr

In [12]:
print(train.apply(lambda col: col.unique()))
print("\n\n")

print(test.apply(lambda col: col.unique()))

surgery                                                          [yes, no]
age                                                         [adult, young]
rectal_temp              [38.1, 37.5, 37.1, 38.0, 39.2, 37.4, 38.3, 39....
pulse                    [132.0, 88.0, 72.0, 52.0, 56.0, 114.0, 48.0, 8...
respiratory_rate         [24.0, 12.0, 30.0, 48.0, 32.0, 36.0, 16.0, 20....
temp_of_extremities                             [cool, cold, normal, warm]
peripheral_pulse                      [reduced, normal, absent, increased]
mucous_membrane          [dark_cyanotic, pale_cyanotic, pale_pink, norm...
capillary_refill_time                          [more_3_sec, less_3_sec, 3]
pain                     [depressed, mild_pain, alert, extreme_pain, se...
peristalsis                      [absent, hypomotile, hypermotile, normal]
abdominal_distention                      [slight, moderate, none, severe]
nasogastric_tube                               [slight, none, significant]
nasogastric_reflux       

Creamers_v test to find the best columns for our model

In [13]:
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
    rcorr = r - ((r - 1)**2) / (n - 1)
    kcorr = k - ((k - 1)**2) / (n - 1)
    return np.sqrt(phi2corr / min((kcorr - 1), (rcorr - 1)))

# Calculate Cramér's V for each column relative to "outcome"
cramer_vs = {}
for column in train.columns[:-1]:  # Exclude the 'outcome' column
    cramer_vs[column] = cramers_v(train[column], train['outcome'])

# Sort the results by Cramér's V in descending order
sorted_cramer_vs = sorted(cramer_vs.items(), key=lambda x: x[1], reverse=True)

# Display the most correlated columns
print(sorted_cramer_vs)

[('lesion_1', 0.37120645659093), ('pain', 0.35888398353368495), ('packed_cell_volume', 0.3482660088021885), ('pulse', 0.34241154610532176), ('total_protein', 0.3325861236373031), ('abdomo_protein', 0.32030538975365996), ('nasogastric_reflux_ph', 0.29646773865893167), ('abdomo_appearance', 0.2843210879484689), ('temp_of_extremities', 0.27489329004867163), ('mucous_membrane', 0.27118375769787734), ('peripheral_pulse', 0.2652365024258519), ('surgical_lesion', 0.24703022943131825), ('capillary_refill_time', 0.24232586874716466), ('abdominal_distention', 0.24063931754186116), ('peristalsis', 0.2235613765013104), ('cp_data', 0.2028915086381601), ('respiratory_rate', 0.19460019329759512), ('abdomen', 0.18808544808140978), ('rectal_exam_feces', 0.18540251460209572), ('rectal_temp', 0.1406845870484661), ('surgery', 0.13627395668799713), ('age', 0.11607309853919205), ('nasogastric_tube', 0.11040238281750323), ('nasogastric_reflux', 0.08070050668567398), ('lesion_3', 0.02822730285849165), ('lesio

In [14]:
# Define the number of top correlated columns to display (e.g., top 3)
top_n = 10

# Display the top N most correlated columns based on Cramér's V
for column, cramers_v_value in sorted_cramer_vs[:top_n]:
    print(f"Column: {column}, Cramér's V: {cramers_v_value}")

Column: lesion_1, Cramér's V: 0.37120645659093
Column: pain, Cramér's V: 0.35888398353368495
Column: packed_cell_volume, Cramér's V: 0.3482660088021885
Column: pulse, Cramér's V: 0.34241154610532176
Column: total_protein, Cramér's V: 0.3325861236373031
Column: abdomo_protein, Cramér's V: 0.32030538975365996
Column: nasogastric_reflux_ph, Cramér's V: 0.29646773865893167
Column: abdomo_appearance, Cramér's V: 0.2843210879484689
Column: temp_of_extremities, Cramér's V: 0.27489329004867163
Column: mucous_membrane, Cramér's V: 0.27118375769787734


Using these top 10 columns for our model

In [15]:
train_1 = train[["lesion_1","pain","packed_cell_volume","pulse","total_protein","abdomo_protein","nasogastric_reflux_ph","abdomo_appearance","temp_of_extremities","mucous_membrane","outcome"]]

Encoding the categorical columns in both test and train datasets


In [16]:
def onehot_encode(train_1, column, categories):
    encoder = OneHotEncoder(categories= categories , drop= "first", sparse=False , dtype=np.int64)
    encoded_features = encoder.fit_transform(train[column].values.reshape(-1,1))
    encoder.transform(train_1[column].values.reshape(-1,1))

    train_1  = train_1.drop(labels=column, axis=1)

    for j, category in enumerate(categories[0][1:]):
        category_column_name = "{}_{}".format(column,category)
        train_1[category_column_name]=encoded_features[:,j]
    return train_1

In [17]:
categories_hotenc = {"temp_of_extremities" : [["normal", "cool", "cold", "warm"]],
                     "mucous_membrane":[["dark_cyanotic","pale_cyanotic","pale_pink","normal_pink","bright_pink","bright_red"]],
                     "pain" : [["depressed","mild_pain","alert","extreme_pain","severe_pain"]],
                     "abdomo_appearance":[["serosanguious","cloudy","clear"]]
                    }

for column, category in categories_hotenc.items():
    train_1= onehot_encode(train_1,column,category)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_outp

In [18]:
train_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 771 entries, 0 to 1233
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   lesion_1                       771 non-null    int64  
 1   packed_cell_volume             771 non-null    float64
 2   pulse                          771 non-null    float64
 3   total_protein                  771 non-null    float64
 4   abdomo_protein                 771 non-null    float64
 5   nasogastric_reflux_ph          771 non-null    float64
 6   outcome                        771 non-null    object 
 7   temp_of_extremities_cool       771 non-null    int64  
 8   temp_of_extremities_cold       771 non-null    int64  
 9   temp_of_extremities_warm       771 non-null    int64  
 10  mucous_membrane_pale_cyanotic  771 non-null    int64  
 11  mucous_membrane_pale_pink      771 non-null    int64  
 12  mucous_membrane_normal_pink    771 non-null    int64  

In [19]:
test_1 = test[["lesion_1","pain","packed_cell_volume","pulse","total_protein","abdomo_protein","nasogastric_reflux_ph","abdomo_appearance","temp_of_extremities","mucous_membrane"]]

In [20]:
test_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824 entries, 0 to 823
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   lesion_1               824 non-null    int64  
 1   pain                   824 non-null    object 
 2   packed_cell_volume     824 non-null    float64
 3   pulse                  824 non-null    float64
 4   total_protein          824 non-null    float64
 5   abdomo_protein         824 non-null    float64
 6   nasogastric_reflux_ph  824 non-null    float64
 7   abdomo_appearance      824 non-null    object 
 8   temp_of_extremities    824 non-null    object 
 9   mucous_membrane        824 non-null    object 
dtypes: float64(5), int64(1), object(4)
memory usage: 64.5+ KB


In [21]:
def onehot_encode(test_1, column, categories):
    categories[0].append(0)
    encoder = OneHotEncoder(categories= categories , drop= "first", sparse=False , dtype=np.int64)
    encoded_features = encoder.fit_transform(test_1[column].values.reshape(-1,1))
    encoder.transform(test_1[column].values.reshape(-1,1))

    test_1  = test_1.drop(labels=column, axis=1)

    for j, category in enumerate(categories[0][1:]):
        category_column_name = "{}_{}".format(column,category)
        test_1[category_column_name]=encoded_features[:,j]
    return test_1

In [22]:
categories_hotenc = {"temp_of_extremities" : [["normal", "cool", "cold", "warm",0]],
                     "mucous_membrane":[["normal_pink","bright_pink","bright_red","dark_cyanotic","pale_pink","pale_cyanotic",0]],
                     "pain" : [["mild_pain","depressed","extreme_pain","severe_pain","alert",0,"moderate"]],
                     "abdomo_appearance":[["clear","serosanguious","cloudy",0]]
                    }

for column, category in categories_hotenc.items():
    test_1= onehot_encode(test_1,column,category)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_outp

In [23]:
test_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824 entries, 0 to 823
Data columns (total 25 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   lesion_1                         824 non-null    int64  
 1   packed_cell_volume               824 non-null    float64
 2   pulse                            824 non-null    float64
 3   total_protein                    824 non-null    float64
 4   abdomo_protein                   824 non-null    float64
 5   nasogastric_reflux_ph            824 non-null    float64
 6   temp_of_extremities_cool         824 non-null    int64  
 7   temp_of_extremities_cold         824 non-null    int64  
 8   temp_of_extremities_warm         824 non-null    int64  
 9   temp_of_extremities_0            824 non-null    int64  
 10  mucous_membrane_bright_pink      824 non-null    int64  
 11  mucous_membrane_bright_red       824 non-null    int64  
 12  mucous_membrane_dark_c

In [24]:

columns_to_drop_test = ['temp_of_extremities_0', 'mucous_membrane_0', 'pain_0', 'pain_moderate','pain_depressed','mucous_membrane_dark_cyanotic','abdomo_appearance_0','abdomo_appearance_serosanguious']
test_1 = test_1.drop(columns=columns_to_drop_test)


columns_to_drop_train = ['pain_mild_pain','mucous_membrane_normal_pink','abdomo_appearance_clear']
train_1 = train_1.drop(columns=columns_to_drop_train)

In [25]:
train_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 771 entries, 0 to 1233
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   lesion_1                       771 non-null    int64  
 1   packed_cell_volume             771 non-null    float64
 2   pulse                          771 non-null    float64
 3   total_protein                  771 non-null    float64
 4   abdomo_protein                 771 non-null    float64
 5   nasogastric_reflux_ph          771 non-null    float64
 6   outcome                        771 non-null    object 
 7   temp_of_extremities_cool       771 non-null    int64  
 8   temp_of_extremities_cold       771 non-null    int64  
 9   temp_of_extremities_warm       771 non-null    int64  
 10  mucous_membrane_pale_cyanotic  771 non-null    int64  
 11  mucous_membrane_pale_pink      771 non-null    int64  
 12  mucous_membrane_bright_pink    771 non-null    int64  

In [26]:
test_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824 entries, 0 to 823
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   lesion_1                       824 non-null    int64  
 1   packed_cell_volume             824 non-null    float64
 2   pulse                          824 non-null    float64
 3   total_protein                  824 non-null    float64
 4   abdomo_protein                 824 non-null    float64
 5   nasogastric_reflux_ph          824 non-null    float64
 6   temp_of_extremities_cool       824 non-null    int64  
 7   temp_of_extremities_cold       824 non-null    int64  
 8   temp_of_extremities_warm       824 non-null    int64  
 9   mucous_membrane_bright_pink    824 non-null    int64  
 10  mucous_membrane_bright_red     824 non-null    int64  
 11  mucous_membrane_pale_pink      824 non-null    int64  
 12  mucous_membrane_pale_cyanotic  824 non-null    int

Performing feature engineering using LOG TRANSFORMATION

In [27]:


# Create an empty DataFrame to store the updated columns
updated_train = pd.DataFrame()

# List of columns to apply the transformation to

columns_to_transform = ['lesion_1', 'packed_cell_volume', 'pulse', 'total_protein', 'abdomo_protein', 'nasogastric_reflux_ph', 'temp_of_extremities_cool', 'temp_of_extremities_cold', 'temp_of_extremities_warm', 'mucous_membrane_pale_cyanotic', 'mucous_membrane_pale_pink', 'mucous_membrane_bright_pink', 'mucous_membrane_bright_red', 'pain_alert', 'pain_extreme_pain', 'pain_severe_pain',  'abdomo_appearance_cloudy']

# Apply the transformation to each column in the list and add them to the updated_train DataFrame
for column in columns_to_transform:
    updated_train[column + '_updated'] = train_1[column] ** (1/2)


In [28]:


# Create an empty DataFrame to store the updated columns for test_1
updated_test = pd.DataFrame()

# List of columns to apply the transformation to for test_1

columns_to_transform_test = ['lesion_1', 'packed_cell_volume', 'pulse', 'total_protein', 'abdomo_protein', 'nasogastric_reflux_ph', 'temp_of_extremities_cool', 'temp_of_extremities_cold', 'temp_of_extremities_warm', 'mucous_membrane_bright_pink', 'mucous_membrane_bright_red', 'mucous_membrane_pale_pink', 'mucous_membrane_pale_cyanotic', 'pain_extreme_pain', 'pain_severe_pain', 'pain_alert', 'abdomo_appearance_cloudy']

# Apply the transformation to each column in the list and add them to the updated_test DataFrame
for column in columns_to_transform_test:
    updated_test[column + '_updated'] = test_1[column] ** (1/2)


In [29]:
print(updated_train.info())
print("/n/n")
print(updated_test.info())

<class 'pandas.core.frame.DataFrame'>
Index: 771 entries, 0 to 1233
Data columns (total 17 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   lesion_1_updated                       771 non-null    float64
 1   packed_cell_volume_updated             771 non-null    float64
 2   pulse_updated                          771 non-null    float64
 3   total_protein_updated                  771 non-null    float64
 4   abdomo_protein_updated                 771 non-null    float64
 5   nasogastric_reflux_ph_updated          771 non-null    float64
 6   temp_of_extremities_cool_updated       771 non-null    float64
 7   temp_of_extremities_cold_updated       771 non-null    float64
 8   temp_of_extremities_warm_updated       771 non-null    float64
 9   mucous_membrane_pale_cyanotic_updated  771 non-null    float64
 10  mucous_membrane_pale_pink_updated      771 non-null    float64
 11  mucous_mem

In [30]:
# Get the column order from updated_train
column_order = updated_train.columns

# Reorganize the columns in updated_test
updated_test = updated_test[column_order]


In [31]:
print(updated_train.info())
print("/n/n")
print(updated_test.info())

<class 'pandas.core.frame.DataFrame'>
Index: 771 entries, 0 to 1233
Data columns (total 17 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   lesion_1_updated                       771 non-null    float64
 1   packed_cell_volume_updated             771 non-null    float64
 2   pulse_updated                          771 non-null    float64
 3   total_protein_updated                  771 non-null    float64
 4   abdomo_protein_updated                 771 non-null    float64
 5   nasogastric_reflux_ph_updated          771 non-null    float64
 6   temp_of_extremities_cool_updated       771 non-null    float64
 7   temp_of_extremities_cold_updated       771 non-null    float64
 8   temp_of_extremities_warm_updated       771 non-null    float64
 9   mucous_membrane_pale_cyanotic_updated  771 non-null    float64
 10  mucous_membrane_pale_pink_updated      771 non-null    float64
 11  mucous_mem

In [32]:
updated_train['outcome'] = train_1['outcome'].copy()

Spliting X and Y for the model

In [33]:
X = updated_train.drop(columns = ["outcome"])

In [34]:
Y = updated_train["outcome"]
Y = Y.map({'died': 0, 'euthanized': 1, 'lived': 2})

Training and testing our model

In [35]:
mode_acc, mode_pred = list(), list()

sk = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 1, random_state = 42)
for i, (train_idx, test_idx) in enumerate(sk.split(X, Y)):

    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y,
        test_size=0.2,
        random_state= 42,
        shuffle=True
        )
    #X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    #Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]

    print('----------------------------------------------------------')

#logistic
    logit_md = make_pipeline(StandardScaler(), LogisticRegression(multi_class = 'multinomial')).fit(X_train, Y_train)

    logit_pred = logit_md.predict(X_test)
    logit_score = f1_score(Y_test, logit_pred, average = 'micro')

    print('Fold', i, '==> Logistic oof F1 score is ==>', logit_score)

    logit_pred_test = logit_md.predict(updated_test)

#LDA

    lda_md = make_pipeline(StandardScaler(), LinearDiscriminantAnalysis()).fit(X_train, Y_train)

    lda_pred = lda_md.predict(X_test)
    lda_score = f1_score(Y_test, lda_pred, average = 'micro')

    print('Fold', i, '==> LDA oof F1 score is ==>', lda_score)

    lda_pred_test = lda_md.predict(updated_test)

#Random Forest

    RF_md = RandomForestClassifier(n_estimators = 500,
                                   max_depth = 5,
                                   min_samples_split = 15,
                                   min_samples_leaf = 10).fit(X_train, Y_train)

    RF_pred = RF_md.predict(X_test)
    RF_score = f1_score(Y_test, RF_pred, average = 'micro')

    print('Fold', i, '==> RF oof F1 score is ==>', RF_score)

    RF_pred_test = RF_md.predict(updated_test)

#Extra trees

    ET_md = ExtraTreesClassifier(n_estimators = 500,
                                 max_depth = 7,
                                 min_samples_split = 15,
                                 min_samples_leaf = 10).fit(X_train, Y_train)

    ET_pred = ET_md.predict(X_test)
    ET_score = f1_score(Y_test, ET_pred, average = 'micro')

    print('Fold', i, '==> ET oof F1 score is ==>', ET_score)

    ET_pred_test = ET_md.predict(updated_test)

#gradient boost

    GB_md = GradientBoostingClassifier(n_estimators = 500,
                                       learning_rate = 0.01,
                                       max_depth = 7,
                                       min_samples_split = 15,
                                       min_samples_leaf = 10).fit(X_train, Y_train)

    GB_pred = GB_md.predict(X_test)
    GB_score = f1_score(Y_test, GB_pred, average = 'micro')

    print('Fold', i, '==> GB oof F1 score is ==>', GB_score)

    GB_pred_test = GB_md.predict(updated_test)


#histgrad

    hist_md = HistGradientBoostingClassifier(l2_regularization = 0.01,
                                             early_stopping = False,
                                             learning_rate = 0.01,
                                             max_iter = 500,
                                             max_depth = 7,
                                             max_bins = 255,
                                             min_samples_leaf = 5,
                                             max_leaf_nodes = 5).fit(X_train, Y_train)

    hist_pred = hist_md.predict(X_test)
    hist_score = f1_score(Y_test, hist_pred, average = 'micro')

    print('Fold', i, '==> Hist oof F1 score is ==>', hist_score)

    hist_pred_test = hist_md.predict(updated_test)

#LGBM

    LGBM_md = LGBMClassifier(objective = 'multiclass',
                             n_estimators = 500,
                             max_depth = 7,
                             learning_rate = 0.01,
                             num_leaves = 20,
                             reg_alpha = 3,
                             reg_lambda = 3,
                             subsample = 0.7,
                             colsample_bytree = 0.7).fit(X_train, Y_train)

    lgb_pred = LGBM_md.predict(X_test)
    lgb_score = f1_score(Y_test, lgb_pred, average = 'micro')

    print('Fold', i, '==> LGBM oof F1 score is ==>', lgb_score)

    lgb_pred_test = LGBM_md.predict(updated_test)

#XGB

    XGB_md = XGBClassifier(objective = 'multi:softprob',
                           tree_method = 'hist',
                           colsample_bytree = 0.7,
                           gamma = 2,
                           learning_rate = 0.01,
                           max_depth = 7,
                           min_child_weight = 10,
                           n_estimators = 500,
                           subsample = 0.7).fit(X_train, Y_train)

    xgb_pred = XGB_md.predict(X_test)
    xgb_score = f1_score(Y_test, xgb_pred, average = 'micro')

    print('Fold', i, '==> XGB oof F1 score is ==>', xgb_score)

    xgb_pred_test = XGB_md.predict(updated_test)

#CatBoost

    Cat_md = CatBoostClassifier(loss_function = 'MultiClass',
                                iterations = 500,
                                learning_rate = 0.01,
                                depth = 7,
                                random_strength = 0.5,
                                bagging_temperature = 0.7,
                                border_count = 30,
                                l2_leaf_reg = 5,
                                verbose = False,
                                task_type = 'CPU').fit(X_train, Y_train)

    cat_pred = Cat_md.predict(X_test)
    cat_score = f1_score(Y_test, cat_pred, average = 'micro')

    print('Fold', i, '==> CatBoost oof F1 score is ==>', cat_score)

    cat_pred_test = Cat_md.predict(updated_test)

#modenamble

    md_preds = pd.concat([pd.Series(logit_pred.flatten()),
                          pd.Series(lda_pred.flatten()),
                          pd.Series(RF_pred.flatten()),
                          pd.Series(ET_pred.flatten()),
                          pd.Series(GB_pred.flatten()),
                          pd.Series(hist_pred.flatten()),
                          pd.Series(lgb_pred.flatten()),
                          pd.Series(xgb_pred.flatten()),
                          pd.Series(cat_pred.flatten())], axis = 1)

    md_preds_test = pd.concat([pd.Series(logit_pred_test.flatten()),
                               pd.Series(lda_pred_test.flatten()),
                               pd.Series(RF_pred_test.flatten()),
                               pd.Series(ET_pred_test.flatten()),
                               pd.Series(GB_pred_test.flatten()),
                               pd.Series(hist_pred_test.flatten()),
                               pd.Series(lgb_pred_test.flatten()),
                               pd.Series(xgb_pred_test.flatten()),
                               pd.Series(cat_pred_test.flatten())], axis = 1)


    mode_ens = mode(md_preds, axis = 1, keepdims = True)[0]
    mode_score = f1_score(Y_test, mode_ens, average = 'micro')

    print('Fold', i, '==> Mode Ensemble oof F1 score is ==>', mode_score)

    mode_acc.append(mode_score)
    mode_pred.append(mode(md_preds_test, axis = 1, keepdims = True)[0])

----------------------------------------------------------
Fold 0 ==> Logistic oof F1 score is ==> 0.6838709677419355
Fold 0 ==> LDA oof F1 score is ==> 0.6903225806451613
Fold 0 ==> RF oof F1 score is ==> 0.7096774193548389
Fold 0 ==> ET oof F1 score is ==> 0.6903225806451613
Fold 0 ==> GB oof F1 score is ==> 0.6967741935483871
Fold 0 ==> Hist oof F1 score is ==> 0.7354838709677418
Fold 0 ==> LGBM oof F1 score is ==> 0.7677419354838709
Fold 0 ==> XGB oof F1 score is ==> 0.7290322580645161
Fold 0 ==> CatBoost oof F1 score is ==> 0.7419354838709677
Fold 0 ==> Mode Ensemble oof F1 score is ==> 0.7225806451612903
----------------------------------------------------------
Fold 1 ==> Logistic oof F1 score is ==> 0.6838709677419355
Fold 1 ==> LDA oof F1 score is ==> 0.6903225806451613
Fold 1 ==> RF oof F1 score is ==> 0.7225806451612903
Fold 1 ==> ET oof F1 score is ==> 0.6967741935483871
Fold 1 ==> GB oof F1 score is ==> 0.6967741935483871
Fold 1 ==> Hist oof F1 score is ==> 0.7354838709677

CREATING OUR SUBMISSION FILE FOR KAGGLE COMPETITION

In [36]:
submission = pd.read_csv('/kaggle/input/sample-data/sample_submission.csv')

In [37]:
submission['outcome'] = mode(np.concatenate(mode_pred, axis = 1), axis = 1, keepdims = True)[0]
submission['outcome'] = submission['outcome'].map({0: 'died', 1: 'euthanized', 2: 'lived'})
submission.to_csv('modeling_submission.csv', index = False)
submission.head()

,id,outcome
0,1235,lived
1,1236,died
2,1237,lived
3,1238,euthanized
4,1239,lived
